In [909]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [910]:
#iris = load_iris()
#df = pd.DataFrame(iris.data, columns=iris.feature_names)
#df['label'] = iris.target
#df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
#data = np.array(df)
#X, y = data[:,:-1], data[:,-1]
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

分类树，计算gini指数作为判断条件。后剪枝的方法。

In [911]:
features = ["年龄", "有工作", "有自己的房子", "信贷情况"]
X_train = pd.DataFrame([
    ["0", "0", "0", "0"],
    ["0", "0", "0", "1"],
    ["0", "1", "0", "1"],
    ["0", "1", "1", "0"],
    ["0", "0", "0", "0"],
    ["1", "0", "0", "0"],
    ["1", "0", "0", "1"],
    ["1", "1", "1", "1"],
    ["1", "0", "1", "2"],
    ["1", "0", "1", "2"],
    ["2", "0", "1", "2"],
    ["2", "0", "1", "1"],
    ["2", "1", "0", "1"],
    ["2", "1", "0", "2"],
    ["2", "0", "0", "0"]
])

X_train = np.array(X_train).astype(np.int32)

y_train = pd.DataFrame(["否", "否", "是", "是", "否", 
                        "否", "否", "是", "是", "是", 
                        "是", "是", "是", "是", "否"])

#print((X_train))

In [912]:
#print(np.array(y_train))
y = np.zeros(len(y_train))
i = 0
for ele in np.array(y_train):
    if ele == '是':
        y[i] = 1
    else:
        y[i] = -1
    i += 1
#print(y)

In [913]:
#print(X_train, y)
dataSet = np.c_[X_train, y]
X, X_test, y_train, y_test = train_test_split(X_train, y, test_size=0.2)
#print(X)
#print(np.shape(dataSet))
#print(X_test, y_test)
dataSet = np.c_[X, y_train]
testSet = np.c_[X_test,y_test]
print(dataSet,"\n", testSet)

[[ 1.  0.  1.  2.  1.]
 [ 0.  1.  0.  1.  1.]
 [ 1.  0.  1.  2.  1.]
 [ 2.  1.  0.  1.  1.]
 [ 0.  1.  1.  0.  1.]
 [ 0.  0.  0.  0. -1.]
 [ 0.  0.  0.  0. -1.]
 [ 1.  1.  1.  1.  1.]
 [ 0.  0.  0.  1. -1.]
 [ 2.  1.  0.  2.  1.]
 [ 1.  0.  0.  1. -1.]
 [ 1.  0.  0.  0. -1.]] 
 [[ 2.  0.  1.  1.  1.]
 [ 2.  0.  0.  0. -1.]
 [ 2.  0.  1.  2.  1.]]


In [914]:
def calcGini(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet:
        currentLabel = featVec[-1]
       # print(currentLabel)
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    Gini = 1.0
    for key in labelCounts:
        prob = float(labelCounts[key] / numEntries)
        Gini -= prob*prob
    return Gini
        

In [915]:
calcGini(dataSet)

0.48611111111111094

In [916]:
def calcGiniWithFeat(dataSet, feature, value):
    D0 = []; D1 = []
    for featVec in dataSet:
        if featVec[feature] == value:
            D0.append(featVec)
        else:
            D1.append(featVec)
    Gini = len(D0) / len(dataSet) * calcGini(D0) + len(D1) / len(dataSet) * calcGini(D1)
    return Gini

In [917]:
calcGiniWithFeat(dataSet, 0, 1)

0.4857142857142858

In [918]:
#print(len(features))



选择最好的特征进行分割，根据计算的gini，选择最小的gini对应的特征。

In [919]:
def chooseBestSplit(dataSet):
    numFeatures = len(features)
    bestGini = np.Inf; bestFeat = 0; bestValue = 0; newGini = 0
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)
        for splitVal in uniqueVals:
            newGini = calcGiniWithFeat(dataSet, i , splitVal)
            if newGini < bestGini:
                bestGini = newGini
                bestFeat = i
    return bestFeat


In [920]:
chooseBestSplit(dataSet)

1

In [921]:
import operator 
def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.items(),key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

In [922]:
# 满足特征条件的子数据集
def splitDataSet(dataSet, bestFeat, value):
    retDataSet = []
    for featVec in dataSet:
        if featVec[bestFeat] == value:
            #print("vec",featVec)
            reducedFeatVec = featVec[:bestFeat]
            temp_list = list(reducedFeatVec)
            #print(featVec[bestFeat+1:])
            temp_list.extend(featVec[bestFeat+1:])
            #print(temp_list)
            retDataSet.append(temp_list)
    retDataSet = np.array(retDataSet)
    return retDataSet
            

In [923]:
res = splitDataSet(dataSet, 1, 0)
print(type(res))

<class 'numpy.ndarray'>


In [924]:
def CreateTree(dataSet, labels):
    classList = [example[-1] for example in dataSet]
    # 只有一种分类
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    # 只有一种特征
    if len(dataSet[0]) == 1:
        return majorityCnt(classList)
    bestFeat = chooseBestSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
   # print("bestFeat:",bestFeat, " bestlabel:",bestFeatLabel)
    myTree = {bestFeatLabel:{}}
    np.delete(labels,bestFeat)
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]
        myTree[bestFeatLabel][value] = CreateTree(splitDataSet(dataSet, bestFeat, value), subLabels)
       # print("subTree:", myTree[bestFeatLabel][value])
    return myTree

In [925]:
ans = CreateTree(dataSet, features)

In [926]:
ans

{'有工作': {0.0: {'有工作': {0.0: -1.0, 1.0: 1.0}}, 1.0: 1.0}}

In [927]:
def classify(inputTree, featLabels, testData):
    print(inputTree)
    firstStr = list(inputTree.keys())[0]
    print(firstStr, type(firstStr))
    secondDict = inputTree[firstStr]
    #print("second:",secondDict)
    featIndex = featLabels.index(firstStr)
    key = testData[featIndex]
    print("testData:",testData)
    print("featIndex:",featIndex, "key:",key)
    value0Feat = secondDict[key]
    print("value0Feat:",value0Feat)
    if isinstance(value0Feat, dict):
        classLabel = classify(value0Feat, featLabels, testData)
    else:
        classLabel = value0Feat
    return classLabel

In [928]:
#classify(ans, features, testSet[0])

In [929]:
def calcTestErr(myTree, testData, labels):
    print("input:",myTree)
    errorCount = 0
    for i in range(len(testData)):
        if classify(myTree, labels,testData[i]) != testData[i][-1]:
            errorCount += 1
    print("未剪枝错误：",errorCount)
    return float(errorCount)

In [930]:
def isTree(obj):
    return (type(obj).__name__ == 'dict')

In [931]:
def testMajor(major, dataTest):
    error = 0.0
    for i in range(len(dataTest)):
        if major != dataTest[i][-1]:
            error += 1
    print("剪枝后错误：",error)
    return float(error)

In [932]:
import copy
#def pruningTree(inputTree, miniGain, evaluationFunc = calcGini):
def pruningTree(inputTree, dataSet, testData, labels):
    print("label:",labels)
    firstStr = list(inputTree.keys())[0]
    secondDict = inputTree[firstStr]
    classList = [example[-1] for example in dataSet]
    featKey = copy.deepcopy(firstStr)
    labelIndex = labels.index(featKey)
    subLabels = copy.deepcopy(labels)
    print("featKey:",featKey, "type:", type(subLabels))
    #(labels, labelIndex)
    for key in list(secondDict.keys()):
        if isTree(secondDict[key]):
            subDataSet = splitDataSet(dataSet, labelIndex, key)
            subTestSet = splitDataSet(testData, labelIndex, key)
            print("subDataSet:",subDataSet)
            if (len(subDataSet) > 0 and len(subTestSet) > 0):
                inputTree[firstStr][key] = pruningTree(secondDict[key], subDataSet,subTestSet, copy.deepcopy(labels))
    if calcTestErr(inputTree,testData,subLabels) < testMajor(majorityCnt(classList), testData):
        return inputTree
    else:
        return majorityCnt(classList)
    

In [933]:
pruningTree(ans, dataSet, testSet, features)

label: ['年龄', '有工作', '有自己的房子', '信贷情况']
featKey: 有工作 type: <class 'list'>
subDataSet: [[ 1.  1.  2.  1.]
 [ 1.  1.  2.  1.]
 [ 0.  0.  0. -1.]
 [ 0.  0.  0. -1.]
 [ 0.  0.  1. -1.]
 [ 1.  0.  1. -1.]
 [ 1.  0.  0. -1.]]
label: ['年龄', '有工作', '有自己的房子', '信贷情况']
featKey: 有工作 type: <class 'list'>
input: {'有工作': {0.0: -1.0, 1.0: 1.0}}
{'有工作': {0.0: -1.0, 1.0: 1.0}}
有工作 <class 'str'>
testData: [2. 1. 1. 1.]
featIndex: 1 key: 1.0
value0Feat: 1.0
{'有工作': {0.0: -1.0, 1.0: 1.0}}
有工作 <class 'str'>
testData: [ 2.  0.  0. -1.]
featIndex: 1 key: 0.0
value0Feat: -1.0
{'有工作': {0.0: -1.0, 1.0: 1.0}}
有工作 <class 'str'>
testData: [2. 1. 2. 1.]
featIndex: 1 key: 1.0
value0Feat: 1.0
未剪枝错误： 0
剪枝后错误： 2.0
input: {'有工作': {0.0: {'有工作': {0.0: -1.0, 1.0: 1.0}}, 1.0: 1.0}}
{'有工作': {0.0: {'有工作': {0.0: -1.0, 1.0: 1.0}}, 1.0: 1.0}}
有工作 <class 'str'>
testData: [2. 0. 1. 1. 1.]
featIndex: 1 key: 0.0
value0Feat: {'有工作': {0.0: -1.0, 1.0: 1.0}}
{'有工作': {0.0: -1.0, 1.0: 1.0}}
有工作 <class 'str'>
testData: [2. 0. 1. 1. 1.]
featI

1.0